In [1]:
!pip install ethernetip

     |████████████████████████████████| 177 kB 1.7 MB/s eta 0:00:01


# coding=utf-8
from scapy.all import *
import binascii
import struct
import time

import crcmod
import libnum

from scapy.layers.inet import IP, UDP
from scapy.layers.l2 import Ether


import sys
sys.path.append('/home/xalanaseon/scapy-cip-enip')
# insert at 1, 0 is the script path (or '' in REPL)
#sys.path.insert(1, '/home/xalanaseon/scapy-cip-enip')

import cip

rawpkt = binascii.unhexlify(
    '801d9cc8bde7001d9cc672e808004500005e2f95400080064746c0a80164'
    'c0a8010ac203af128e7a438701bd1e5e5018829c2a07000070001e000200'
    '1600000000000000000000000000000000000000000000000200a1000400'
    '2042b5ffb1000a008a074c0320b225002200')
pkt = Ether(rawpkt)
pkt.show()

import numpy as np

plc_cube = []
hmi_cube = []

def packet_analysis(pkt):
    if TCP in pkt:
        #if len(pkt[TCP].payload) > 6:
        pkt.show()
        #d_time = float(str(pkt.time))
        #if(pkt[TCP].dport == 502):
        #    plc_cube.append(d_time)
        #else:
        #    hmi_cube.append(d_time)


try:
    sniff(offline='DataSet/Dec2019_00000_20191206100500.pcap', prn=packet_analysis, count=1000)
    #plc_cube = np.asarray(plc_cube)
    #hmi_cube = np.asarray(hmi_cube)
    print('Completed')

except Exception as sniff_dec_err:
    traceback.print_exc()
    print(str(sniff_dec_err))

In [ ]:
import ethernetip
import random
import socket
import struct
import time


def main():
    hostname = "192.168.1.32"
    broadcast = "192.168.255.255"
    inputsize = 1
    outputsize = 1
    EIP = ethernetip.EtherNetIP(hostname)
    C1 = EIP.explicit_conn(hostname)

    listOfNodes = C1.scanNetwork(broadcast, 5)
    print("Found ", len(listOfNodes), " nodes")
    for node in listOfNodes:
        name = node.product_name.decode()
        sockinfo = ethernetip.SocketAddressInfo(node.socket_addr)
        ip = socket.inet_ntoa(struct.pack("!I", sockinfo.sin_addr))
        print(ip, " - ", name)

    pkt = C1.listID()
    if pkt is not None:
        print("Product name: ", pkt.product_name.decode())

    pkt = C1.listServices()
    print("ListServices:", str(pkt))

    # read input size from global system object (obj 0x84, attr 4)
    r = C1.getAttrSingle(0x84, 1, 4)
    if 0 == r[0]:
        print("Read CPX input size from terminal success (data: " + str(r[1]) + ")")
        inputsize = struct.unpack("B", r[1])[0]

    # read output size from global system object (obj 0x84, attr 5)
    r = C1.getAttrSingle(0x84, 1, 5)
    if 0 == r[0]:
        print("Read CPX output size from terminal sucess (data: " + str(r[1]) + ")")
        outputsize = struct.unpack("B", r[1])[0]

    # configure i/o
    print("Configure with {0} bytes input and {1} bytes output".format(inputsize, outputsize))
    EIP.registerAssembly(ethernetip.EtherNetIP.ENIP_IO_TYPE_INPUT, inputsize, 101, C1)
    EIP.registerAssembly(ethernetip.EtherNetIP.ENIP_IO_TYPE_OUTPUT, outputsize, 100, C1)
    EIP.startIO()

    C1.registerSession()

    C1.setAttrSingle(ethernetip.CIP_OBJ_TCPIP, 1, 6, "fbxxx")

    for i in range(1, 8):
        r = C1.getAttrSingle(ethernetip.CIP_OBJ_IDENTITY, 1, i)
        if 0 == r[0]:
            print("read ok attr (" + str(i) + ") data: " + str(r[1]))
        else:
            print("Err: " + str(r[0]))

    C1.sendFwdOpenReq(101, 100, 1)
    C1.produce()

    while True:
        try:
            time.sleep(0.2)
            C1.outAssem[random.randint(0, len(C1.outAssem) - 1)] = True
            C1.outAssem[random.randint(0, len(C1.outAssem) - 1)] = False
        except KeyboardInterrupt:
            break
    C1.stopProduce()
    C1.sendFwdCloseReq(101, 100, 1)
    EIP.stopIO()


if __name__ == "__main__":
    main()

In [29]:
from scapy.all import *
import binascii
import struct
import time

import crcmod
import libnum

import dpkt
from dpkt.ip import IP, IP_PROTO_UDP
from dpkt.udp import UDP
from dpkt.tcp import TCP

f = open('DataSet/Dec2019_00000_20191206100500.pcap','rb')
pcap = dpkt.pcap.Reader(f)
count = 0
for ts, buf in pcap:
    print(ts, len(buf))
    #print('Ethernet Frame: ', eth.src, eth.dst, eth.type)
    eth = dpkt.ethernet.Ethernet(buf)
    ip = eth.data
    #print(ip)
    if type(ip.data) == TCP : 
        tcp = ip.data
        print('sequences number ',tcp.seq)
        print(binascii.hexlify(bytes(tcp.data)))
    count = count + 1
    if count > 100:
        break

1575597904.3746 134
sequences number  1538011731
b'7000340001005400000000000000000000000000000000000000000000000200a1000400586ab2ffb10020007a264d09910f484d495f504c414e545f524553455400a002db48010000000000'
1575597904.374601 124
sequences number  1538011807
b'70002e0001005400000000000000000000000000000000000000000000000200a10004007e56b2ffb1001a0015264c0a9111484d495f50335f5045524d495353495645000100'
1575597904.374602 128
sequences number  1538011807
b'70002e0001005400000000000000000000000000000000000000000000000200a10004007e56b2ffb1001a0015264c0a9111484d495f50335f5045524d495353495645000100'
1575597904.374603 280
sequences number  3718062355
b'7000ca0001005900000000000000000000000000000000000000000000000200a100040050050080b100b6000a4fcc000000010020232c24000020420000c842000070410000204165a46641000000000004000040400000604000000000000000002000007a44000096440000484400007a43e0c75f441301001302001301001301000b020001ffffffffab8aa54300000000ab8aa5430b010001010000000000d970d9406a941b4302006dfb0c4200